In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [6]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [11]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

<class 'script.Script'>
<class 'script.Script'>


.
----------------------------------------------------------------------
Ran 1 test in 0.254s

OK


In [12]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [ ]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

In [13]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [14]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.198s

OK


## My new address

In [6]:
# create another address , new
from ecc import PrivateKey
from helper import hash256, little_endian_to_int

secret = little_endian_to_int(hash256(b'edu little ultra secret'))
private_key = PrivateKey(secret)

address = private_key.point.address(compressed=True, testnet=True)
print(address)

mzBKFS9qB99RfwGtNupG9zBftfWoQ8BWa1


## adress new 2

In [8]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int

secret = little_endian_to_int(hash256(b'edu little ultra secret for a second time'))
private_key = PrivateKey(secret)

address = private_key.point.address(compressed=True, testnet=True)
print(address)

n2tCd4ydaDccWvL3FQg3v8VhfdPpETVjfg


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [10]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
secret = little_endian_to_int(hash256(b'edu little ultra secret for a second time'))
private_key = PrivateKey(secret)

# change address should be the address generated from Chapter 4
change_address = "n2tCd4ydaDccWvL3FQg3v8VhfdPpETVjfg"

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx = bytes.fromhex("df682dbb44c86c5a669df1e773d17920350f512ff95686fb358c77ef7d01311f")
prev_index = 1
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
input_1 = TxIn(prev_tx, prev_index)
tx_ins = []
tx_ins.append(input_1)

prev_tx_amount = 0.000345

# target amount should be 60% of the output amount
target_amount = 0.000345 * 0.6
# set the fee to some reasonable amount
fee = 0.00000285
# change amount = amount from the prev tx - target amount - fee
change_amount = prev_tx_amount - target_amount - fee

# create a transaction output for the target amount and address
tx_outs = []

h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*100_000_000)
output_1 = TxOut(target_satoshis, script_pubkey)

h160 = decode_base58(change_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*100_000_000)
output_2 = TxOut(change_satoshis, script_pubkey)

# create a transaction output for the change amount and address
tx_outs.append(output_1)
tx_outs.append(output_2)

# create the transaction object
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# sign the one input in the transaction object using the private key
sign = tx_obj.sign_input(0, private_key)
print("sign input 1:")
print(sign)
# print the transaction's serialization in hex
print(tx_obj.serialize().hex())
# broadcast at http://testnet.blockchain.info/pushtx

sign input 1:
True
01000000011f31017def778c35fb8656f92f510f352079d173e7f19d665a6cc844bb2d68df010000006a47304402200389ebb14d045eff0333cca9d9265e94ae88a085ed36e60a045e168f70f95156022069caa5f11896c97cce3443ca391d8d338099a42abecf0306778b99570dcf70d3012103845fa8a598dcd0355fb972ae19c8685b8ba39bdbf02c834d70df57280b7409e9ffffffff02dc500000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788accb340000000000001976a914ea5f5641ff719e3ffbe217183feb06b28ace32e288ac00000000


see tx in [mempool](#)

### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

set up a second tx in faucet
0.000155 coins sent to n2tCd4ydaDccWvL3FQg3v8VhfdPpETVjfg

tx id: 7dfe12ccb6990f7a440af4497ebf122bae98dfa4f48a63d0896fc4c5de86c88b

## chpater 4 original
tx id: df682dbb44c86c5a669df1e773d17920350f512ff95686fb358c77ef7d01311f
recived 0.00034500 tBTC


In [22]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'n2tCd4ydaDccWvL3FQg3v8VhfdPpETVjfg'

# get the private key from the exercise in Chapter 4
secret = little_endian_to_int(hash256(b'edu little ultra secret for a second time'))
private_key = PrivateKey(secret)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# first

# input 1
prev_tx_1 = bytes.fromhex("7dfe12ccb6990f7a440af4497ebf122bae98dfa4f48a63d0896fc4c5de86c88b")
prev_index_1 = 0

# create the first transaction input with the default ScriptSig and
# sequence
input_1 = TxIn(prev_tx_1, prev_index_1)

tx_ins = []
tx_ins.append(input_1)

# input 2
# get the prev_tx and prev_index from the transaction in Exercise 4
prev_tx_2 = bytes.fromhex("4d597d77558ef15818a7a7912feae381212e3b266fd79a0112a227c71153c47d")
prev_index_2 = 1
# create the second transaction input with the default ScriptSig and
# sequence
input_2 = TxIn(prev_tx_2, prev_index_2)
tx_ins.append(input_2)

# set the fee to some reasonable amount
fee_amount = 0.00000345
# fee_satoshis = int(fee_amount*100_000_000)
# target amount should be the sum of the inputs - fee
# target_amount_satoshis = input_1.value(testnet=True) + input_2.value(testnet=True) - fee_satoshis
target_amount = 0.00015500 + 0.00013515  - fee_amount # 0.0002867
target_satoshis = int(target_amount * 100_000_000)
# create a transaction output for the amount and address
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
# target_satoshis = int(target_amount*100000000)
output_1 = TxOut(target_satoshis, script_pubkey)

tx_outs.append(output_1)

tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# sign the first input using the private key
print("sign_input_1:", tx_obj.sign_input(0, private_key))
# sign the second input using the private key 
print("sign_input_2:", tx_obj.sign_input(1, private_key))

# print the transaction's serialization in hex
print("Tx in hex: ", tx_obj.serialize().hex())
# broadcast at http://testnet.blockchain.info/pushtx

sign_input_1: True
sign_input_2: True
Tx in hex:  01000000028bc886dec5c46f89d0638af4a4df98ae2b12bf7e49f40a447a0f99b6cc12fe7d000000006a47304402203f0d5443fe6ad04c4fa317c17685a815bb513adcf9baa864d1d55f7139b76e5502202ae7d62a74349f5ce03dbfc12e3ed4ec51b1056c3d968e7d52b5d6a046be0466012103845fa8a598dcd0355fb972ae19c8685b8ba39bdbf02c834d70df57280b7409e9ffffffff7dc45311c727a212019ad76f263b2e2181e3ea2f91a7a71858f18e55777d594d010000006a473044022037f717d3f7f324973902dd5bde0800123d4edf5821bdd933d63ee596c420e07b02206dc986063ff080f8b9903dfeb04eca1ac82db5a4c312845ea32781d90b640cf6012103845fa8a598dcd0355fb972ae19c8685b8ba39bdbf02c834d70df57280b7409e9ffffffff01fd6f0000000000001976a914ea5f5641ff719e3ffbe217183feb06b28ace32e288ac00000000


ee Tx in [mempool](https://mempool.space/testnet/tx/8a96cee8d4a6e96b3bc130dbf8382f56c3592a4d43636c67463ffa0dc232560a)

tx id: 8a96cee8d4a6e96b3bc130dbf8382f56c3592a4d43636c67463ffa0dc232560a

My address in [mempoool](https://mempool.space/testnet/address/n2tCd4ydaDccWvL3FQg3v8VhfdPpETVjfg)